In [ ]:
#XlsxWriter is a Python module for creating Excel XLSX files.
pip install xlsxwriter

     |████████████████████████████████| 148 kB 14.3 MB/s 


In [ ]:
import numpy as np
from numpy.core.fromnumeric import mean
from google.colab import files
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

stocks = pd.read_csv('sp_500_stocks.csv')
from secretly import IEX_CLOUD_API_TOKEN

In [ ]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [ ]:
#symbol = 'AAPL'
#api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
#data = requests.get(api_url).json()
#print(data)

#dividing the list of stocks in batches of 100

def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [ ]:
#creating batch api call

hqm_columns = [
                'Ticker',
                'Price',
                'One-Year Price Return',
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol,
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ],
                                                  index = hqm_columns),
                                        ignore_index = True)

hqm_dataframe


,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,159.83,0.515159,N/A,0.165024,N/A,0.041671,N/A,-0.0807042,N/A,N/A
1,AAL,19.48,0.548422,N/A,-0.0904251,N/A,-0.09393,N/A,-0.102139,N/A,N/A
2,AAP,240.10,0.53092,N/A,0.17065,N/A,0.0778097,N/A,0.0696021,N/A,N/A
3,AAPL,155.20,0.312327,N/A,0.114668,N/A,0.00213701,N/A,0.0118946,N/A,N/A
4,ABBV,109.10,0.35758,N/A,-0.00320136,N/A,-0.0713591,N/A,0.0265963,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,127.58,0.271726,N/A,0.0801924,N/A,0.0303128,N/A,0.00348721,N/A,N/A
501,ZBH,157.87,0.0675428,N/A,-0.150821,N/A,-0.0556513,N/A,0.000270678,N/A,N/A
502,ZBRA,551.89,0.756208,N/A,0.0707275,N/A,-0.029445,N/A,-0.0488079,N/A,N/A
503,ZION,66.69,1.07435,N/A,0.242582,N/A,0.310996,N/A,0.0992413,N/A,N/A


In [ ]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

hqm_dataframe.fillna(0,inplace=True)
# pd.set_option('display.max_columns', None)


for row in hqm_dataframe.index:
    for time_period in time_periods:

        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)*10


hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)


,Ticker,Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
148,DVN,41.69,3.556241,1,0.961918,0.99802,0.613397,0.99802,0.235102,0.986139,9.95545
317,MRO,16.95,2.879295,0.99604,0.612743,0.990099,0.481808,0.994059,0.319341,1,9.9505
175,FANG,111.32,2.964575,0.99802,0.465233,0.976238,0.409861,0.990099,0.234923,0.984158,9.87129
357,OXY,35.26,2.585495,0.994059,0.430939,0.974257,0.329105,0.970297,0.258498,0.99604,9.83663
109,COP,78.83,1.478750,0.978218,0.574767,0.986139,0.392844,0.986139,0.181272,0.964356,9.78713
...,...,...,...,...,...,...,...,...,...,...,...
308,MKTX,422.20,-0.279869,0.0019802,-0.198857,0.0356436,-0.146792,0.039604,-0.039855,0.140594,0.544554
246,IPGP,159.79,-0.210307,0.0118812,-0.304419,0.00792079,-0.264083,0.0019802,-0.028213,0.192079,0.534653
291,LW,59.40,-0.187530,0.0178218,-0.290298,0.0118812,-0.229106,0.00594059,-0.039880,0.138614,0.435644
377,PNW,67.90,-0.197990,0.0138614,-0.191271,0.0435644,-0.207848,0.00990099,-0.076529,0.0514851,0.29703


In [ ]:
#Generating Excel File

writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

# Creating the Formats We'll Need For Our .xlsx File

background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['One-Year Price Return', percent_template],
                    'D': ['One-Year Return Percentile', percent_template],
                    'E': ['Six-Month Price Return', percent_template],
                    'F': ['Six-Month Return Percentile', percent_template],
                    'G': ['Three-Month Price Return', percent_template],
                    'H': ['Three-Month Return Percentile', percent_template],
                    'I': ['One-Month Price Return', percent_template],
                    'J': ['One-Month Return Percentile', percent_template],
                    'K': ['HQM', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

"""## Saving Our Excel Output

As before, saving our Excel output is very easy:
"""

writer.save()

files.download('momentum_strategy.xlsx')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>